In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split as rsplit
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os
import torchvision.models
from torchsummary import summary

torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
torch.backends.cudnn.deterministic = True
np.random.seed(1)



In [ ]:
class ImageFolderWithPaths(ImageFolder):
    
    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
            
        return img, target, path
    
class DatasetFolderWithPaths(torchvision.datasets.DatasetFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
            
        return sample, target, path


In [ ]:
def get_data_loader(batch_size=8, sets=["train", "val", "test"], tlearning=""):
    data_loader = []
    
#     data_transform = transforms.Compose( [transforms.CenterCrop(224), transforms.Resize((224,224)) , transforms.ToTensor()])
    data_transform = transforms.Compose([transforms.Resize((224,224)) , transforms.ToTensor()])
    
    # Load dataset based on the passed in folder name & whether it is transfer learning or not.
    for s in sets:
        if tlearning == "anet":
            print("\n #### Loading AlexNet Features ####\n")
            data_set  = DatasetFolderWithPaths("weather_dataset/{}".format("anet_{}".format(s)),  
                                                           loader=torch.load, extensions=('.tensor'))
        elif tlearning =="vgg16":
            print("\n #### Loading VGG16 Features ####\n")
#             data_set  = torchvision.datasets.DatasetFolder("weather_dataset/{}".format("vgg16_{}".format(s)), 
#                                                           loader=torch.load, extensions=('.tensor'))
            data_set  = DatasetFolderWithPaths("weather_dataset/{}".format("vgg16_{}".format(s)), 
                                                          loader=torch.load, extensions=('.tensor'))
        else:
            data_set = ImageFolderWithPaths("weather_dataset/{}".format(s),
                               transform=data_transform)
        
        print("Complete Dataset:\n ", data_set)
        data_loader.append(torch.utils.data.DataLoader(data_set, batch_size=batch_size, shuffle=True))
    
    return data_loader

In [ ]:
def get_model_path(name, bs, lr, data_set, total_epoch, epoch, csv=False, tlearning=""):
    if csv:
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(name, bs, lr, data_set, total_epoch)
        if tlearning == "anet":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(name,bs, lr, "anet_{}".format(data_set),total_epoch)
        elif tlearning == "vgg16":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(name,bs, lr, "vgg16_{}".format(data_set),total_epoch)
    else:
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/epoch_{}".format(name, bs, lr, data_set, total_epoch,epoch)
        if tlearning == "anet":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/epoch_{}".format(name,bs, lr, "anet_{}".format(data_set),epoch)
        elif tlearning == "vgg16":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/epoch_{}".format(name,bs, lr, "vgg16_{}".format(data_set),total_epoch)    
    return model_path

In [ ]:
def save_feats(sets = ["train", "val", "test"], tlearning="anet"):
    root_path = "weather_dataset/"
    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    pretrained_net = ""
    subpath = ""
    if tlearning == "anet":
        print("Saving as anet features")
        pretrained_net = torchvision.models.alexnet(pretrained=True)
        subpath = "anet"
    elif tlearning =="vgg16":
        print("Saving as vgg16 features")
        pretrained_net = torchvision.models.vgg16(pretrained=True)
        subpath = "vgg16"
    else:
        assert 0
    
    data_loader = get_data_loader(batch_size=1, sets=sets)
    
    
    for i, data in enumerate(data_loader):
        path = "{}/{}_{}".format(root_path, subpath,sets[i])
        
        
        if not os.path.isdir(path):
            os.mkdir(path)
        
#         n = 0
        
        for images, labels, im_path in data:
            im_path = im_path[0].split('\\')[-1]
#             print(images, labels, im_path)
#             break
            features = pretrained_net.features(images)
            features_tensor = torch.from_numpy(features.detach().numpy())
            folder = "{}/{}/".format(path, classes[labels])
            
            if not os.path.isdir(folder):
                os.mkdir(folder)
                
#             torch.save(features_tensor.squeeze(0), '{}/{}.tensor'.format(folder, str(n)))
            torch.save(features_tensor.squeeze(0), '{}/{}.tensor'.format(folder, im_path))
#             n+=1
    

In [ ]:
def get_accuracy(model=None, img_net=None, meta_net=None, data_loader=None, dset="train",conf_matrix=False):
    # calculate total correct predictions within the specified data_set

#     classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny', 'z-other']
    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    x = np.zeros(shape=(len(classes), len(classes)))
#     print(x.shape)


#     img_net = load_IMG_Network("vgg16CNN_BS_1024_LR_0.015_dataset_vgg16_train_numepochs_20/epoch_5")
#     meta_net = load_META_Network("MetadataModel/epoch_199")
    
    mdata = m.get_available_metadata(dset)
         

    l = 0
    correct, total = 0 , 0
    for images, labels, path in data_loader:
        images = images.cuda()
        labels = labels.cuda()

        
        batch_ids = [i.split('\\')[-1].split('.')[0] for i in path]
        batch_ids_bool = [str(i) in mdata for i in batch_ids]


        if not all(batch_ids_bool):
            assert(0)

        meta_feats = [list(mdata[i][0].values()) for i in batch_ids]
        meta_feats = torch.FloatTensor(meta_feats)
        meta_feats = meta_feats.cuda()
        

        img_out = img_net(images)
        metadata_out = meta_net(meta_feats)
        
        img_metadata_in = torch.cat((img_out, metadata_out), 1)

        
        outputs = model(img_metadata_in)
        
        pred = outputs.max(1, keepdim=True)[1]
        
        if conf_matrix:
            i = labels.view_as(pred).item()
            j = pred.item()
            x[i][j] += 1
            
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += images.shape[0]
        
        l+=1
    if conf_matrix:
        return (correct / total), x
    else:
        return (correct / total)

In [ ]:
def get_accuracy_2net(model=None, img_net=None, meta_net=None, data_loader=None, dset="train",conf_matrix=False):
    # calculate total correct predictions within the specified data_set

#     classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny', 'z-other']
    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    x = np.zeros(shape=(len(classes), len(classes)))
#     print(x.shape)


#     img_net = load_IMG_Network("vgg16CNN_BS_1024_LR_0.015_dataset_vgg16_train_numepochs_20/epoch_5")
#     meta_net = load_META_Network("MetadataModel/epoch_199")
    
    mdata = m.get_available_metadata(dset)
         
        
    image_network_fullnet = vgg16_CNN().cuda()
    state = torch.load("vgg16CNN_BS_1024_LR_0.015_dataset_vgg16_train_numepochs_20/epoch_5")
    image_network_fullnet.load_state_dict(state)


    l = 0
    correct, total = 0 , 0
    for images, labels, path in data_loader:
        images = images.cuda()
        labels = labels.cuda()

        
        batch_ids = [i.split('\\')[-1].split('.')[0] for i in path]
        batch_ids_bool = [str(i) in mdata for i in batch_ids]
        

        outputs = 0
        if not all(batch_ids_bool):
#             print(images)
            outputs = image_network_fullnet(images)
        
        else:
            meta_feats = [list(mdata[i][0].values()) for i in batch_ids]
            meta_feats = torch.FloatTensor(meta_feats)
            meta_feats = meta_feats.cuda()


            img_out = img_net(images)
            metadata_out = meta_net(meta_feats)

            img_metadata_in = torch.cat((img_out, metadata_out), 1)
    
#             print(img_metadata_in)
            outputs = model(img_metadata_in)
        
        
        pred = outputs.max(1, keepdim=True)[1]
        
        if conf_matrix:
            i = labels.view_as(pred).item()
            j = pred.item()
            x[i][j] += 1
            
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += images.shape[0]
        
        l+=1
    if conf_matrix:
        return (correct / total), x
    else:
        return (correct / total)

In [ ]:
 def train_net(net, img_net=None, meta_net=None,batch_size=8, lr=0.01, num_epochs=20, data_set=["train", "val"], weight_decay=0, tlearning=""):
    
    model_path=""
    
    if tlearning == "anet":
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(net.name,batch_size, lr, "anet_{}".format(data_set[0]),num_epochs)
    elif tlearning == "vgg16":
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(net.name,batch_size, lr, "vgg16_{}".format(data_set[0]),num_epochs)
    else:
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(net.name,batch_size, lr, data_set[0],num_epochs)

    
    os.mkdir(model_path)
    
    
    with open('./{}network_params.txt'.format(model_path),'w') as file:    
        print(net.parameters, file=file)
    
    
    data_loader = get_data_loader(batch_size=batch_size, sets=data_set, tlearning=tlearning)
    train_loader, val_loader = data_loader
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    
    train_acc, train_loss = [], []
    val_acc, val_loss = [], []
    iters = []
    
#     img_net = load_IMG_Network("vgg16CNN_BS_1024_LR_0.015_dataset_vgg16_train_numepochs_20/epoch_5")
#     meta_net = load_META_Network("MetadataModel/epoch_199")
    
    train_data = m.get_available_metadata("train")
#     print(train_data.keys())
#     print("3984038" in train_data.keys())
    
    for epoch in range(num_epochs):
        print("Epoch: ", epoch)
        
        batch_total =0.0
        
        # go through data loaded in batches of batch_size
        # i.e. # of data loaded / batch_size should be number of iterations
        
        j = 0
        total_loss = 0
        for i, data in enumerate(train_loader, 0):              
            images, labels, path = data
            
            images = images.cuda()
            labels = labels.cuda()
                     
#             print(path)
            batch_ids = [i.split('\\')[-1].split('.')[0] for i in path]
            batch_ids_bool = [str(i) in train_data for i in batch_ids]
#            print(batch_ids)
#            print(batch_ids_bool)
            
            if not all(batch_ids_bool):
                assert(0)
            
            x = [list(train_data[i][0].values()) for i in batch_ids]
            x = torch.FloatTensor(x)
            x = x.cuda()
#             print(x, x.shape)
#             print(labels, labels.shape)
#             print(images, images.shape)
            
            img_out = img_net(images)
            metadata_out = meta_net(x)
            
            
#             print(img_out)
#             print(metadata_out)
            
            img_metadata_in = torch.cat((img_out, metadata_out), 1)

#             print(img_metadata_in, img_metadata_in.shape)
            
            outputs = net(img_metadata_in)
#             print(outputs.shape)
            loss = criterion(outputs, labels)
            loss.backward()
            total_loss+=float(loss)
            j+=images.shape[0]
            optimizer.step()
            optimizer.zero_grad()

            pred = outputs.max(1, keepdim=True)[1]
            
#             print(pred)
            mini_batch_correct = pred.eq(labels.view_as(pred)).sum().item()
            mini_batch_total = images.shape[0]
#             train_acc.append(mini_batch_correct / mini_batch_total)

            print("Epoch {}, Mini Batch Accuracy {:.2f}%, Loss {:.2f}%".format(epoch ,mini_batch_correct/mini_batch_total * 100 , total_loss/j * 100))
            
#             n+=1
        
        iters.append(epoch)
        train_acc.append(get_accuracy(net, img_net,meta_net,train_loader, "train"))
        val_acc.append(get_accuracy(net, img_net,meta_net, val_loader, "val"))
        train_loss.append(total_loss/j)
        
        print("\nEpoch {} Summary: Train Accuracy {:.2f}%, Validation Accuracy {:.2f}%, Loss {:.2f}%\n".format(epoch,
                                                                                                          train_acc[-1] * 100,
                                                                                                          val_acc[-1] * 100,
                                                                                                          total_loss/j * 100))
        
        torch.save(net.state_dict(), model_path + "epoch_{}".format(str(epoch)))           



    
    np.savetxt("{}train_acc.csv".format(model_path), train_acc)
    np.savetxt("{}train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}val_acc.csv".format(model_path), val_acc)
    np.savetxt("{}iters.csv".format(model_path), iters)
    
    return iters, train_loss, val_acc, train_acc

# Network Definition

In [ ]:
# class vgg16_CNN(nn.Module):
#     def __init__(self):
#         self.name="vgg16CNN"
#         super(vgg16_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(512, 400, 2)
#         self.fc1 = nn.Linear(400*6*6, 64)
#         self.fc2 = nn.Linear(64, 5)
        
#     def forward(self, img):
#         x = F.relu(self.conv1(img))
#         x = x.view(-1, 400*6*6)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

    
class vgg16_CNN(nn.Module):
    def __init__(self):
        self.name="vgg16CNN"
        super(vgg16_CNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(512, 400, 2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(400*6*6, 64),
            nn.ReLU(),
            nn.Linear(64, 5)
        )

        
    def forward(self, img):
        x = self.encoder(img)
        return x

    
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.name="ANN"
        self.encoder = nn.Sequential(
            nn.Linear(5, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 5)
        )


    def forward(self, x):
        x = self.encoder(x)
        return x  


# class ANN(nn.Module):
#     def __init__(self):
#         super(ANN, self).__init__()
#         self.name="ANN"
#         self.encoder = nn.Sequential(
#             nn.Linear(4, 30),
#             nn.Tanh(),
#             nn.Linear(30, 30),
#             nn.Tanh(),
#             nn.Linear(30, 30),
#             nn.Tanh(),
#             nn.Linear(30, 5)
#         )



#     def forward(self, x):
#         x = self.encoder(x)
#         return x


class ANN_Class(nn.Module):
    def __init__(self):
        super(ANN_Class, self).__init__()
        self.name="ANN_Class"
        self.encoder = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 5)
        )
    
    def forward(self,x):
        x = self.encoder(x)
        return x

In [ ]:
def plot_graphs(iters, train_loss, train_acc, val_acc):
    plt.title("Training Curve")
    plt.plot(iters, train_loss, label="Train")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Training")
    plt.plot(iters, val_acc, label="Validation")    
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

In [ ]:
def print_conf_matrix(conf_matrix):
    print("\n\n")

    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']

    for i in range(len(classes)):
        if i == 0:
            print('', end='\t')

        print(classes[i], end = '\t')

    print("\tTotal Samples")
    print('\n')

    for i in range(len(classes)):

        print(classes[i], end ='\t')
        for j in range(len(classes)):
            print(conf_matrix[i][j], end='\t')
        print("\t", sum(conf_matrix[i, :]))
        print('\n')
    
    for i in range(len(classes)):
        if i == 0:
            print("Guesses", end="\t")
        
        print(sum(conf_matrix[:,i]),  end ='\t')
        
    print("\n\n")
    
    for i in range(len(classes)):
        print("{} : {:.2f}%".format(classes[i],conf_matrix[i][i]/sum(conf_matrix[i, :]) * 100))
    
    

# AlexNet

In [ ]:
# Can save the features into your drive to avoid running this again
#save_feats(sets = ["train", "val", "test"],tlearning="anet") #if using alexnet transfer learning

# VGG16

In [ ]:
# Can save the features into your drive to avoid running this again
#save_feats(sets = ["train", "val", "test"],tlearning="vgg16") #if using alexnet transfer learning

In [ ]:
# train_net(net, batch_size=128, lr=0.001, num_epochs=10, data_set=["train", "val"], weight_decay=0, tlearning="vgg16")

In [ ]:
def load_IMG_Network(model_path):
    model = vgg16_CNN().cuda()
    state = torch.load(model_path)
    model.load_state_dict(state)
#     print(model)
#     summary(model, (512, 7, 7))

    modified_model = list(model.children())[0][:-2]
    model = nn.Sequential(*modified_model)
#     print(model)
#     summary(model, (512, 7, 7))
    return model

# MetaData

In [ ]:
def load_META_Network(model_path):
    
    model = ANN().cuda()
    state = torch.load(model_path)
    model.load_state_dict(state)
#     print(model)
#     summary(model, (1, 5))


    modified_model = list(model.children())[0][:-2]
    model = nn.Sequential(*modified_model)
#     print(model)
#     summary(model, (1, 5))
    return model

In [ ]:
import meta_parser
m = meta_parser.metadata_map()

#  3 Network Training

In [ ]:
model = ANN_Class().cuda()
img_net = load_IMG_Network("vgg16CNN_BS_1024_LR_0.015_dataset_vgg16_train_numepochs_20/epoch_5")
meta_net = load_META_Network("MetadataModel/epoch_199")
iters, train_loss, val_acc, train_acc = train_net(model, img_net=img_net, meta_net=meta_net,num_epochs=50, lr=1e-2, batch_size=1024, data_set=["train","val"], tlearning="vgg16")

In [ ]:
plot_graphs(iters, train_loss, train_acc, val_acc)

In [ ]:
epoch_idx = val_acc.index(max(val_acc))
print(iters[epoch_idx],train_acc[epoch_idx],max(val_acc))

# Model Loader

In [ ]:
model = ANN_Class().cuda()
state = torch.load("ANN_Class_BS_1024_LR_0.01_dataset_vgg16_train_numepochs_50/epoch_35")
model.load_state_dict(state)

img_net = load_IMG_Network("vgg16CNN_BS_1024_LR_0.015_dataset_vgg16_train_numepochs_20/epoch_5")
meta_net = load_META_Network("MetadataModel/epoch_199")

In [ ]:
data_loader = get_data_loader(batch_size=1, sets=["val"], tlearning="vgg16")
x, y = get_accuracy(model,img_net,meta_net, data_loader[0], "val",conf_matrix=True)

In [ ]:
print(x)
print_conf_matrix(y)

In [ ]:
data_loader = get_data_loader(batch_size=1, sets=["test"], tlearning="vgg16")
x, y = get_accuracy_2net(model,img_net,meta_net, data_loader[0], "test",conf_matrix=True)

In [ ]:
print(x)
print_conf_matrix(y)